In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/ifajcik/efficientqa_pruned/efficientQA_submission'

In [2]:
!ls 

 configurations
 Dockerfile
 download_data.sh
 efficientqa_eval
 efficientqa_input
 efficientqa_output
 fit_qa
 fusion
 language
 NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_abstractive_reader_outputs.json
 NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_fused.json
 NQ-open.efficientqa.dev.1.1.no-annotations.jsonlfused_w_abs.json
 NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_fused_with_r_rr.json
 NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_gen_reranked_reader_outputs.json
'NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_preprocessed_queryenc_wikipassages_NQ_open_val_<class '\''transformers.tokenization_roberta.RobertaTokenizer'\''>.json'
 NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_ranked_outputs.jsonl
 NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reader_outputs.json
 NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reranked_outputs.jsonl
 NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reranked_outputs.jsonl_fuseindecoder_preprocessed_for_C25_t5-base_L512.jso

In [3]:
fused_dataf = "NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_fused_with_r_rr.json"
abstractive_data = "NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_abstractive_reader_outputs.json"
gt_file = "efficientqa_eval/NQ-open.efficientqa.dev.1.1.jsonl"

In [4]:
import json
import jsonlines

with open(fused_dataf) as f:
    er = json.load(f)
with open(abstractive_data) as f:
    gr = json.load(f)
correct = {x['question']:x['answer'] for x in list(jsonlines.open(gt_file))}

In [5]:
def argmax(l):
    f = lambda i: l[i]
    return max(range(len(l)), key=f)

In [6]:
import fit_qa.scripts.common.evaluate_predictions as eval_utils
from fit_qa.scripts.common.evaluate_predictions import exact_match_score

def evaluate(preds, gts):
    assert len(preds) == len(gts)
    hits = 0
    for q, a in preds.items():
        gt_answers = gts[q]
        hits+= int(eval_utils.metric_max_over_ground_truths(
            metric_fn=exact_match_score, prediction=a, ground_truths=gt_answers))
    print(f"Accuracy {hits*100./len(preds):.5f}, ({hits}/{len(preds)})")
    
e_predictions = {e['raw_question'] :e['answers'][argmax(e['reader_scores'])]   for e in er.values()}
a_predictions = {e['raw_question'] :e['answers'][argmax(e['reader_scores'])]   for e in gr.values()}    
evaluate(e_predictions,correct)
evaluate(a_predictions,correct)

Accuracy 47.66667, (858/1800)
Accuracy 43.50000, (783/1800)


### Create dataset only from borderline cases. e.g. when one is correct and other one is not

In [7]:

labels,X,Y = [], [], [] 

for e_p, a_p in zip(er.values(), gr.values()):
    assert e_p['raw_question'] == a_p['raw_question']
    
    
    correct_answers = correct[e_p['raw_question']]
    
    
    ep_best_score_idx  = argmax(e_p['reader_scores'])
    ep_best_score = e_p['reader_scores'][ep_best_score_idx]
    ep_prediction = e_p['answers'][ep_best_score_idx]
    ep_correct= bool(eval_utils.metric_max_over_ground_truths(
        metric_fn=exact_match_score, prediction=ep_prediction, ground_truths=correct_answers))
    
    ap_best_score_idx  = argmax(a_p['reader_scores'])
    ap_best_score = a_p['reader_scores'][ap_best_score_idx]
    ap_prediction = a_p['answers'][ap_best_score_idx]
    ap_correct= bool(eval_utils.metric_max_over_ground_truths(
        metric_fn=exact_match_score, prediction=ap_prediction, ground_truths=correct_answers))
    
    if ep_correct and not ap_correct:
        labels.append(1)
    elif not ep_correct and ap_correct:
        labels.append(0)
    else:
        continue # skip non-borderline cases
        
    X.append(ep_best_score)
    Y.append(ap_best_score)
 

Create test dataset

In [8]:

labels_test,X_test,Y_test = [], [], [] 
BOTH_HIT = 2
NONE_HIT= 3
for e_p, a_p in zip(er.values(), gr.values()):
    assert e_p['raw_question'] == a_p['raw_question']
    
    
    correct_answers = correct[e_p['raw_question']]
    
    
    ep_best_score_idx  = argmax(e_p['reader_scores'])
    ep_best_score = e_p['reader_scores'][ep_best_score_idx]
    ep_prediction = e_p['answers'][ep_best_score_idx]
    ep_correct= bool(eval_utils.metric_max_over_ground_truths(
        metric_fn=exact_match_score, prediction=ep_prediction, ground_truths=correct_answers))
    
    ap_best_score_idx  = argmax(a_p['reader_scores'])
    ap_best_score = a_p['reader_scores'][ap_best_score_idx]
    ap_prediction = a_p['answers'][ap_best_score_idx]
    ap_correct= bool(eval_utils.metric_max_over_ground_truths(
        metric_fn=exact_match_score, prediction=ap_prediction, ground_truths=correct_answers))
    
    if ep_correct and not ap_correct:
        labels_test.append(1)
    elif not ep_correct and ap_correct:
        labels_test.append(0)
           
    elif ep_correct and ap_correct:
        labels_test.append(BOTH_HIT)
    elif not ep_correct and not ap_correct:
        labels_test.append(NONE_HIT)
        
    X_test.append(ep_best_score)
    Y_test.append(ap_best_score)

In [9]:
import torch
class ConstrainedLR(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn((1), requires_grad=True))
        self.b = torch.nn.Parameter(torch.randn((1), requires_grad=True))
        self.bias = torch.nn.Parameter(torch.randn((1), requires_grad=True))
        
    def forward(self,X,Y):
        r = self.a*X + self.b*Y + self.bias
        return r

In [10]:
import torch
labels = torch.FloatTensor(labels)
X = torch.FloatTensor(X)
Y = torch.FloatTensor(Y)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)
X[:10], Y[:10], labels[:10]

(tensor([ 2.5139, -0.6049,  0.8458, -1.5533,  1.3244,  0.6597,  1.3970,  0.7529,
          0.3308,  0.1160]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([1., 1., 1., 0., 1., 1., 1., 0., 0., 0.]))

In [11]:
len(X)

213

In [12]:
import torch.nn.functional as F
F.binary_cross_entropy_with_logits(X+Y, labels)

tensor(0.7081)

In [13]:
def evaluate_model(preds, labels):
    assert len(preds) == len(labels)
    hits=0
    total=len(labels)
    for p,l in zip(preds.tolist(),labels):
        if l==BOTH_HIT:
            hits+=1
        elif l==NONE_HIT:
            pass
        else:
            hits+=int(p==l)
    return hits/total*100

In [14]:
import torch.nn.functional as F
from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup

def run_training():
    STEPS=1000
    model =  ConstrainedLR()
    model = model.train()
    opt = torch.optim.SGD(model.parameters(), lr=0.1)
    scheduler = get_linear_schedule_with_warmup(
                    opt,
                    num_warmup_steps=10,
                    num_training_steps=STEPS
                )

    best_acc= 0
    best_r = None
    iterator = tqdm(range(STEPS))

    for i in iterator:
        logits = model(X, Y)
        l_list = F.binary_cross_entropy_with_logits(logits, labels, reduction='none')
        l = l_list.mean()
        l.backward()
        opt.step()
        scheduler.step()
        opt.zero_grad()
        if i % 1 == 0:
            r = { k: v.item() for k,v in dict(model.named_parameters()).items()}
            model.eval()
            acc = evaluate_model((model(X_test,Y_test)>0).int(),labels_test)
            model.train()
            if acc>best_acc:
                iterator.set_description(str(acc))
                best_acc = acc
                best_r = r
    return best_acc, best_r

total_best_acc, total_best_r = 0.,None
for _ in range(50):
    acc,r = run_training()
    if acc>total_best_acc:
        total_best_acc=acc
        total_best_r=r
        print(r)

48.333333333333336:  19%|█▉        | 189/1000 [00:00<00:00, 1883.32it/s]

{'a': 0.45572715997695923, 'b': 1.580994725227356, 'bias': -1.2058194875717163}


48.27777777777777: 100%|██████████| 1000/1000 [00:00<00:00, 1731.33it/s]


e.g.:
```
47.77777777777778 {'a': 0.5522062182426453, 'b': -0.7874355912208557, 'bias': 2.0110630989074707}
```

In [15]:
print(total_best_acc, total_best_r)

48.333333333333336 {'a': 0.45572715997695923, 'b': 1.580994725227356, 'bias': -1.2058194875717163}


In [16]:
model =  ConstrainedLR()
best = total_best_r
best_sd = {k:torch.FloatTensor([v]) for k,v in best.items()}
model.load_state_dict(best_sd)
evaluate_model((model(X_test,Y_test)>0).int(),labels_test)

48.333333333333336